__MIDTERM 1__

Full name: Nguyễn Thị Minh Ngọc
<br>
Student ID: 11219280
<br>
Class: DSEB 63

In [1]:
import pandas as pd
import numpy as np
import regex as re

# Part 1: Basic Data Cleaning

1. Read the `listings.csv` file

In [2]:
listings = pd.read_csv("midterm1-data/listings.csv")
listings.head()

FileNotFoundError: [Errno 2] No such file or directory: 'midterm1-data/listings.csv'

2. Clean "available" column: replaces all "y" and "Y" by "yes" and all "n" and "No" by "no"

In [ ]:
# Define unique values of "available" column
listings['available'].unique()

array(['Y', 'yes', 'n', 'no', 'y', 'NO'], dtype=object)

In [ ]:
listings['available'] = listings['available'].apply(lambda x: 'yes' if x.lower().startswith('y') else 'no')
listings.head()

,id,address,num_rooms,available
0,7332.0,"369 Barbara Square\nWhiteberg, MT 64361",__4,yes
1,NaN,"720 Stephen Garden Apt. 138\nSouth Haroldland,...",4,yes
2,7023.0,"PSC 9388, Box 5285\nAPO AA 33803",3,no
3,2024.0,"93507 Wells Brook Suite 898\nScottfort, IL 03739",(5),yes
4,6396.0,"35641 Ashley Burg Suite 422\nScottview, SD 80942",__6,no


3. Remove all non-digit characters from the num_rooms column

In [ ]:
listings['num_rooms'] = listings['num_rooms'].str.extract('(\d+)')
listings.head()

,id,address,num_rooms,available
0,7332.0,"369 Barbara Square\nWhiteberg, MT 64361",4,yes
1,NaN,"720 Stephen Garden Apt. 138\nSouth Haroldland,...",4,yes
2,7023.0,"PSC 9388, Box 5285\nAPO AA 33803",3,no
3,2024.0,"93507 Wells Brook Suite 898\nScottfort, IL 03739",5,yes
4,6396.0,"35641 Ashley Burg Suite 422\nScottview, SD 80942",6,no


# Part 2: String Manipulation

In [ ]:
hacker_news = pd.read_csv("midterm1-data/hacker_news.csv")
hacker_news.head()

,id,title,url,num_points,num_comments,author,created_at
0,12224879,Interactive Dynamic Video,http://www.interactivedynamicvideo.com/,386,52,ne0phyte,8/4/2016 11:52
1,11964716,Florida DJs May Face Felony for April Fools' W...,http://www.thewire.com/entertainment/2013/04/f...,2,1,vezycash,6/23/2016 22:20
2,11919867,Technology ventures: From Idea to Enterprise,https://www.amazon.com/Technology-Ventures-Ent...,3,1,hswarna,6/17/2016 0:01
3,10301696,Note by Note: The Making of Steinway L1037 (2007),http://www.nytimes.com/2007/11/07/movies/07ste...,8,2,walterbell,9/30/2015 4:12
4,10482257,Title II kills investment? Comcast and other I...,http://arstechnica.com/business/2015/10/comcas...,53,22,Deinos,10/31/2015 9:48


1. Use a regex pattern and the ignorecase flag to count the number of mentions of SQL in the title column

In [ ]:
hacker_news['title'] = hacker_news['title'].astype('str')

In [ ]:
hacker_news['title'].str.extract('(sql)', re.IGNORECASE).notnull().sum()

0    108
dtype: int64

2. Extract the mentions of different SQL flavors into a new column and clean those duplicates by making them all lowercase

* Create a new dataframe named `hn_sql`, including only rows that mention a SQL flavor

In [ ]:
hn_sql = hacker_news[hacker_news['title'].str.contains('sql', flags=re.IGNORECASE)].copy()
hn_sql.head()

,id,title,url,num_points,num_comments,author,created_at
9,11370829,Crate raises $4M seed round for its next-gen S...,http://techcrunch.com/2016/03/15/crate-raises-...,3,1,hitekker,3/27/2016 18:08
142,10957172,PostgreSQL: Linux VS Windows part 2,http://www.sqig.net/2016/01/postgresql-linux-v...,16,3,based2,1/23/2016 4:21
221,11544342,MemSQL (YC W11) Raises $36M Series C,http://blog.memsql.com/memsql-raises-series-c/,74,14,ericfrenkiel,4/21/2016 18:32
394,10620525,The History of SQL Injection,http://motherboard.vice.com/read/the-history-o...,38,9,kawera,11/24/2015 13:25
419,10301554,Pentesterlab Tutorial SQL injection to web ad...,https://pentesterlab.com/exercises/from_sqli_t...,2,1,pentestercrab,9/30/2015 3:32


* Create a new column called `flavor` in the `hn_sql` dataframe, containing extract mentions of SQL flavors, defined as:
    * Any time SQL is preceded by one or more word characters
    * Ignoring all case variation

In [ ]:
hn_sql['flavor'] = hn_sql['title'].str.extract('(\w+sql)', re.IGNORECASE)
hn_sql['flavor']

9               NaN
142      PostgreSQL
221          MemSQL
394             NaN
419             NaN
            ...    
19133    PostgreSQL
19580    PostgreSQL
19769           NaN
19802    PostgreSQL
19965           NaN
Name: flavor, Length: 108, dtype: object

* Use the `Series.str.lower()` method to clean the values in the `flavor` columns by converting them to lowercase. Assign values back to the column in `hn_sql`

In [ ]:
hn_sql['flavor'] = hn_sql['flavor'].str.lower()
hn_sql['flavor']

9               NaN
142      postgresql
221          memsql
394             NaN
419             NaN
            ...    
19133    postgresql
19580    postgresql
19769           NaN
19802    postgresql
19965           NaN
Name: flavor, Length: 108, dtype: object

* Use the `DataFrame.pivot_tables()` method to create a pivot table, `sql_pivot`:
    * The index of the pivot table should be the flavor column
    * The values of the pivot tables should be the mean of the `num_comments` column, aggregated by SQL flavor

In [ ]:
sql_pivot = hn_sql.pivot_table(index='flavor', values='num_comments', aggfunc='mean')
sql_pivot

,num_comments
flavor,
cloudsql,5.000000
memsql,14.000000
mysql,12.230769
nosql,14.529412
postgresql,25.962963
sparksql,1.000000


3. Extract Domains from URLs

* Use the regular expression pattern to extract the URL components from from url column of the hn dataframe. Assign the results to url_parts. Add names to each group: "protocol", "domain" and "path"

In [ ]:
url_parts = hn_sql['url'].str.extract('(^\S+):').rename(columns={0:'protocol'})

In [ ]:
url_parts = hn_sql['url'].str.extract(r'(?P<protocol>http|https|ftp)://(?P<domain>[^/]+)/(?P<path>.*)')
url_parts.head()

,protocol,domain,path
9,http,techcrunch.com,2016/03/15/crate-raises-4m-seed-round-for-its-...
142,http,www.sqig.net,2016/01/postgresql-linux-vs-windows-part-2.html
221,http,blog.memsql.com,memsql-raises-series-c/
394,http,motherboard.vice.com,read/the-history-of-sql-injection-the-hack-tha...
419,https,pentesterlab.com,exercises/from_sqli_to_shell


# Part 3: GroupBy Operations

In [ ]:
hacker_news['title'].str.lower().str.contains('^ask hn').sum()

1744

In [ ]:
hacker_news['title'].str.lower().str.contains('^show hn').sum()

1162

# Part 4: Duplicate Data

In [ ]:
reve = pd.read_csv("midterm1-data/revenue.csv")
reve

,CustomerID,Revenue,date
0,R1004,50,2021-12-01
1,W1002,5000,2021-09-01
2,R1002,150,2021-08-01
3,W1001,2000,2021-05-01
4,W1001,2200,2021-05-01
5,R1001,100,2021-04-01
6,R1001,30,2020-12-01
7,R1003,270,2020-09-01


In [ ]:
reve = reve.groupby(['CustomerID', 'date']).max().reset_index()
reve = reve[['CustomerID', 'Revenue', 'date']]
reve

,CustomerID,Revenue,date
0,R1001,30,2020-12-01
1,R1001,100,2021-04-01
2,R1002,150,2021-08-01
3,R1003,270,2020-09-01
4,R1004,50,2021-12-01
5,W1001,2200,2021-05-01
6,W1002,5000,2021-09-01


In [ ]:
reve = reve.sort_values(['CustomerID', 'date'])
reve = reve.drop_duplicates(subset=['CustomerID'], keep='last', ignore_index=True)
reve

,CustomerID,Revenue,date
0,R1001,100,2021-04-01
1,R1002,150,2021-08-01
2,R1003,270,2020-09-01
3,R1004,50,2021-12-01
4,W1001,2200,2021-05-01
5,W1002,5000,2021-09-01


# Part 5: Missing Value And Data Transformation

In [ ]:
bank = pd.read_csv("midterm1-data/bank-additional-full.csv",sep=';')
bank

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [ ]:
bank['month'].unique()

array(['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'mar', 'apr',
       'sep'], dtype=object)

1. Encoding month

In [ ]:
# 1. Using apply
convert_month = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9,
                 'oct': 10, 'nov': 11, 'dec': 12}
bank['enc_month1'] = bank['month'].apply(lambda x: convert_month[x])
bank['enc_month1']

0         5
1         5
2         5
3         5
4         5
         ..
41183    11
41184    11
41185    11
41186    11
41187    11
Name: enc_month1, Length: 41188, dtype: int64

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from numpy import asarray

2. Transformation

In [ ]:
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler

power = PowerTransformer(method='yeo-johnson', standardize=False)
minmax = MinMaxScaler()
bank['duration_T'] = power.fit_transform(np.array(bank['duration']).reshape((-1, 1)))
bank['duration_T'] = minmax.fit_transform(np.array(bank['duration_T']).reshape((-1, 1)))
bank['duration_T']

0        0.530975
1        0.459655
2        0.512160
3        0.461292
4        0.552608
           ...   
41183    0.564020
41184    0.582818
41185    0.489278
41186    0.602846
41187    0.519427
Name: duration_T, Length: 41188, dtype: float64